# 🏠 Real Estate Price Analysis - EDA
Bu not defteri, emlak verisi üzerinde keşifsel veri analizi (EDA) yapmak amacıyla hazırlanmıştır.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")

In [ ]:
df = pd.read_csv("real_estate_data_utf8sig.csv")
df.head()

## 📋 Veri Özeti

In [ ]:
df.info()


In [ ]:
df.isnull().sum()

## 📊 Betimsel İstatistikler

In [ ]:
df.describe()

## 🔗 Korelasyon Matrisi

In [ ]:
plt.figure(figsize=(10,6))
sns.heatmap(df.corr(numeric_only=True), annot=True, cmap="coolwarm")
plt.title("Korelasyon Matrisi")
plt.show()

## 📈 Fiyat Dağılımı

In [ ]:
plt.figure(figsize=(10,5))
sns.histplot(df['price'], bins=50, kde=True)
plt.title("Fiyat Dağılımı")
plt.xlabel("Fiyat (TL)")
plt.show()

## 🏘️ Oda Sayısına Göre Ortalama Fiyat

In [ ]:
plt.figure(figsize=(10,5))
sns.barplot(data=df, x="number_of_rooms", y="price", estimator='mean')
plt.title("Oda Sayısına Göre Ortalama Fiyat")
plt.ylabel("Fiyat (TL)")
plt.xlabel("Oda Sayısı")
plt.show()

## 📏 Alan (m²) ile Fiyat Arasındaki İlişki

In [ ]:
plt.figure(figsize=(10,5))
sns.scatterplot(data=df, x="area_sqm", y="price")
plt.title("Alan ve Fiyat İlişkisi")
plt.xlabel("Alan (m²)")
plt.ylabel("Fiyat (TL)")
plt.show()